In [3]:
import requests
import json

In [1]:
APPLICATION_ID=""
BOT_TOKEN=""

In [4]:

url = f"https://discord.com/api/v8/applications/{APPLICATION_ID}/commands"

# This is an example CHAT_INPUT or Slash Command, with a type of 1
json = {
    "name": "starter",
    "description": "Generate a conversation starter given some topics",
    "options": [
        {
            "name": "topics",
            "description": "List of topics separated by ','",
            "type": 3,
            "required": False,
            # "choices": [
            #     {
            #         "name": "Common",
            #         "value": "ice breaker,travel"
            #     },
            #     {
            #         "name": "Scientific",
            #         "value": "philosophy,science,physic,mathematic,biology"
            #     },
            # ]
        },
    ]
}

# For authorization, you can use either your bot token
headers = {
    "Authorization": f"Bot {BOT_TOKEN}"
}

r = requests.post(url, headers=headers, json=json)
r.text

'{"id": "912067168210931783", "application_id": "912000855186751489", "version": "915235565639708723", "default_permission": true, "default_member_permissions": null, "type": 1, "name": "starter", "name_localizations": null, "description": "Generate a conversation starter given some topics", "description_localizations": null, "dm_permission": null, "options": [{"type": 3, "name": "topics", "name_localizations": null, "description": "List of topics separated by \',\'", "description_localizations": null}]}'

In [5]:

url = f"https://discord.com/api/v8/applications/{APPLICATION_ID}/commands"


# For authorization, you can use either your bot token
headers = {
    "Authorization": f"Bot {BOT_TOKEN}"
}


r = requests.get(url, headers=headers)
r.json()

[{'id': '912067168210931783',
  'application_id': '912000855186751489',
  'version': '915235565639708723',
  'default_permission': True,
  'default_member_permissions': None,
  'type': 1,
  'name': 'starter',
  'description': 'Generate a conversation starter given some topics',
  'dm_permission': None,
  'options': [{'type': 3,
    'name': 'topics',
    'description': "List of topics separated by ','"}]}]

In [6]:

url = f"https://discord.com/api/v8/applications/{APPLICATION_ID}/commands/912069384191754310"


# For authorization, you can use either your bot token
headers = {
    "Authorization": f"Bot {BOT_TOKEN}"
}


r = requests.delete(url, headers=headers)
r.text

''

In [9]:
interaction_token = "aW50ZXJhY3Rpb246OTE0MTI5ODc3ODExMjAwMDgwOmtCZVBvV1g5TmtoNmZOYURZNlFzVUNLYk81elRsV2kya0VCN3V5d2dSMWxzdFZ5WGlWTU9tbWpzMjJwV21DTzIwTU1JYzM5eUpLN3hqMmNTdGw5QUlsQjFFaHZlVzNzT1poUllqYVFUcENLdDZLN1lPMEE5OThTM1NOOGNxa0Fr"
# https://discord.com/developers/docs/resources/webhook#execute-webhook
r = requests.post(
    f"https://discord.com/api/v8/webhooks/{APPLICATION_ID}/{interaction_token}",
    headers={
        "Content-Type": "application/json"
    },
    data=json.dumps(
        {
            "content": "foo",
        },
    ),
)
r.text

'{"id": "914130174948294656", "type": 19, "content": "foo", "channel_id": "888438976128638998", "author": {"id": "912000855186751489", "username": "Langame", "avatar": "71c05f3e97fc8b6ebf015aad8e67186d", "discriminator": "8825", "public_flags": 524288, "bot": true}, "attachments": [], "embeds": [], "mentions": [], "mention_roles": [], "pinned": false, "mention_everyone": false, "tts": false, "timestamp": "2021-11-27T12:26:57.425000+00:00", "edited_timestamp": null, "flags": 0, "components": [], "application_id": "912000855186751489", "webhook_id": "912000855186751489", "message_reference": {"channel_id": "888438976128638998", "guild_id": "888438975646285834", "message_id": "914129879199518730"}}'